In [100]:
#Przyklad rzutu monetą jako sprawdzenie hipotezy czy jest ona "uczciwa"

In [101]:
#jako ze kazdy rzut jest probą Bernoulliego a wiec X (liczba wyrzuconych orlow) jest zmienna losowa o rozkladzie 
#dwumianowym (n,p), zgodnie z tym mozemy to przyblizyc za pomoca rozkladu normalnego

In [102]:
from typing import Tuple
import math

def normal_approximation_to_binomial(n: int, p: float) -> Tuple[float, float]:
    """Okresla wartosc mi i sigma"""
    mi = p * n
    sigma = math.sqrt(p * (1 - p) * n)
    return mi, sigma

In [103]:
#w przypadku gdy zmienna losowa charakteryzuje sie rozkladem normalnym, to mozemy skorzystac z funkcji
#normal_cdf aby okreslic prawdopodobienstwo tego, ze dana wartosc znajduje sie w obrebie jakiegos interwalu lub poza nim

In [104]:
from scratch.probability import normal_cdf

In [105]:
from scratch.probability import normal_cdf
# Funkcja dystrubuanty rozkladu normalnego okresla prawdodpobienstwo tego, ze zmienna znajduje sie ponizej wartosci progowej
normal_probability_below = normal_cdf

#jezeli nie znajduje sie nad wartoscia progowa, to znajduje sie pod nia
def normal_probability_above(lo: float,
                            mi: float = 0,
                            sigma: float = 1) -> float:
    """Prawdopodobienstwo tego, ze N(mi, sigma) jest wieksze niz lo."""
    return 1 - normal_cdf(lo, mi, sigma)

#wartosc znajduje sie w przedziale, jezeli jest mniejsza od gornej wartosci granicznej i wieksza od dolnej wartosci granicznej.
def normal_probability_between(lo: float,
                              hi: float,
                              mi: float,
                              sigma: float = 1) -> float:
    """prawdopodobienstwo tego, ze N(mi, sigma) jest pomiedzy lo i hi"""
    return normal_cdf(hi, mi, sigma) - normal_cdf(lo, mi, sigma)

#wartosc jest poza przedzialem, jezeli nie znajduje sie pomiedzy ograniczeniami

def normal_probability_outside(lo: float,
                              hi: float,
                              mi: float = 0,
                              sigma: float = 1) -> float:
    """prawdopodobienstwo tego, ze N(mi, sigma) nie jest pomiedzy lo i hi"""
    return 1 - normal_probability_between(lo, hi, mi, sigma)


In [106]:
#operacja odwrotna do poprzedniej, znajdujemy niekoncowy obszar lub symetryczny interwal wokol sredniej
#musi on odpowiadac pewnemu poziomowi prawdopodobienstwa
#teraz zeby okreslic wysrodkowany interwal na wartosc sredniej i zawierajacy 60% prawdopodobienstwa, to musimy
#okreslic wartosci odciecia w ktorych dolne i gorne krancowe obszary zawieraja po 20% prawdopodobienstwa ( pozostawiamy
#srodkowe 60%)

from scratch.probability import inverse_normal_cdf
def normal_upper_bound(probability: float,
                      mi: float = 0,
                      sigma: float = 1) -> float:
    """zwraca "z" przy zachowaniu warunku P(Z <= z) = prawdopodobienstwo"""
    return inverse_normal_cdf(probability, mi, sigma)

def normal_lower_bound(probability: float,
                      mi: float = 0,
                      sigma: float = 1) -> float:
    """Zwraca "z" przy zachowaniu warunku P(Z >= z) = prawdopodobienstwo"""
    return inverse_normal_cdf(1 - probability, mi, sigma)

def normal_two_sided_bounds(probability: float,
                           mi: float = 0,
                           sigma: float = 1) -> Tuple[float, float]:
    """zwraca granice symetryczne(umieszczone wokol sredniej), ktore obejmuja okreslone prawdopodobienstwo"""
    tail_probability = (1 - probability) / 2
    
    #nad gorna granica powinna znajdowac sie wartosc tail_probability
    upper_bound = normal_lower_bound(tail_probability, mi, sigma)
    
    #pod dolna granica powinna znajdowac sie wartosc tail_probability
    
    lower_bound = normal_upper_bound(tail_probability, mi, sigma)
    
    return lower_bound, upper_bound

In [107]:
mi_0, sigma_0 = normal_approximation_to_binomial(1000, 0.5)
mi_0, sigma_0
#rozklad jest zblizony do rozkladu normalnego

(500.0, 15.811388300841896)

In [108]:
#istotnosc - oznacza jak bardzo jestesmy sklonni popelniac blad typu 1 ("falszywie pozytywny")
#gdzie odrzucamy hipoteze H0, nawet gdy jest prawdziwa
#przyjmuje sie chec popelnienia tego bledu na 5% lub 1%

#(461, 531)
lower_bound, upper_bound = normal_two_sided_bounds(0.95, mi_0, sigma_0)
lower_bound, upper_bound

(469.01026640487555, 530.9897335951244)

In [109]:
#moc testu - prawdopodobienstwo niepopelnienia bledu typu 2
#tutaj nie odrzuca sie hipotezy h0 nawet wtedy, gdy jest ona falszywa

#sprwadzamy co sie dzieje, jezeli p wynosi 0.55(czesciej wypada orzel)

#ograniczenie 95% na podstawie zalozenia ze p jest rowne 0.5
lo, hi = normal_two_sided_bounds(0.95, mi_0, sigma_0)
lo, hi
#rzeczywiste wartosci mi i sigma przy zalozeniu ze p = 0.55
mi_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)
mi_1, sigma_1

#blad typu drugiego oznacza blad polegajacy na nieodrzuceniu hipotezy zerowej,
#do czego dochodzi, gdy X ziac znajduje sie w naszym poczatkowym interwale
type_2_probability = normal_probability_between(lo, hi, mi_1, sigma_1)
power = 1 - type_2_probability
power


0.8865480012953671

In [110]:
#test jednostronny - odrzuca hipoteze zerowa, gdy wartosc x jest wieksza od 500, ale nie wtedy, gdy jest mniejsza od 500.
hi = normal_upper_bound(0.95, mi_0, sigma_0)
hi
#wynosi 526(<531) poniewaz potrzebujemy wiecej prawdopodobienstwa w gornej czesci
type_2_probability = normal_probability_below(hi, mi_1, sigma_1)
power = 1 - type_2_probability
power

0.9363794803307173

# Wartosc p

In [120]:
def two_sided_p_value(x: float, mi: float = 0, sigma: float = 1) -> float:
    """Jak prawdopodobne jest zaobserwowanie wartosci przynajmniej tak skrajnej jak x
    (w obydwu kierunkach) jezeli wartosci sa z przedzialu N(mi, sigma)"""
    
    if x >= mi:
        #jezeli x jest wiekszy od sredniej
        return 2 * normal_probability_above(x, mi, sigma)
    else:
        #jezeli x jest mniejsze od sredniej
        return 2 * normal_probability_below(x, mi, sigma)
    
#jezeli wyrzucimy orla 530 razy to wykonamy nastepujaca funkcje

two_sided_p_value(529.5, mi_0, sigma_0)

#529.5 zamiast 530 bo - poprawka ciaglosci - 

0.06207721579598835

# symulacja - dowodzenie sensownosci powyzszych szacunkow

In [ ]:
# import random as rd
extreme_value_count = 0
for _ in range(1000):
    num_heads = sum(1 if rd.random() < 0.5 else 0
                   for _ in range(1000))
    if num_heads >= 530 or num_heads <= 470:
        extreme_value_count += 1
        
        
    #wartosc p wynosi 0.062, wiec okolo 62 skrajne wartosci z 1000
assert 59 < extreme_value_count < 65 , f'{extreme_value_count}'

# przedziały ufnosci

In [112]:
#w tym przypadku korzystamy z szacunkow, bo nie znamy wartosci p
#czesto stosuje sie to w praktyce
#Korzystajac z przyblizenia o rozkladzie normalnym, dochodzimy do wniosku, ze "jestesmy pewnie na 95%" co do tego
#ze nastepujacy przedzial zawiera przawdziwa wartosc parametru p

p_hat = 525 / 1000
mi = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)
print(sigma)
normal_2_sided = normal_two_sided_bounds(0.95, mi, sigma)
print(normal_2_sided)
#wartosc miesci sie w naszym przedziale ufnosci, moneta jest uczciwa

#gdybysmy zaobserwowali wyrzucenie orla 540 razy to

p_hat = 540 / 1000
mi = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)
print(sigma)

normal_2_sided_2 = normal_two_sided_bounds(0.95, mi, sigma)
print(normal_2_sided_2)
#w takim przypadku moneta nie przechodzi testu - zalozylismy, ze gdyby hipoteza ta byla prawdziwa, to okazywalaby
#sie prawdziwa w 95% przypadkow

0.015791611697353755
(0.4940490278129096, 0.5559509721870904)
0.015760710643876435
(0.5091095927295919, 0.5708904072704082)


# hakowanie wartosci p

In [113]:
#procedura ktora blednie odrzuca hipoteze zerowa w tylko 5% przypadkow, bedzie zgodnei z definicja blednie odrzucac hipoteze
#zerowa przez 5% czasu

from typing import List
def run_experiment()-> List[bool]:
    """wykonaj 1000 rzutow moneta. True oznacza orla, False oznacza reszke"""
    return [rd.random() < 0.5 for _ in range(1000)]

def reject_fairness(experiment: List[bool]) -> bool:
    """zaklada 5% poziom ufnosci"""
    num_heads = len([flip for flip in experiment if flip])
    return num_heads < 469 or num_heads > 531

rd.seed(0)
experiments = [run_experiment() for _ in range(1000)]
num_rejections = len([experiment for experiment in experiments if reject_fairness(experiment)])

assert num_rejections == 46

# przyklad: przeprowadzenie testu A-B

In [122]:
#zakldamy ze Na ludzi widzialo reklame A i na z nich ja kliknelo. Kazde wyswietlenie reklamy mozemy traktowac jako probe
#Bernoulliego, w ktorej pa jest prawdopodobienstwem tego, ze ktos kliknie reklame A. Wtedy (jezeli Na jest wieksze, ale warunek
#ten jest w tym przypadku spelniony) wiemy ze na/Na jest zmienna losowa o rozkladzie zblizonym do normalnego, a wiec o sredniej
#pa i odchyleniu standardowym ...

#tak samo dla nb/Nb ktore to jest zmienna losowa o rozkladzie zblizonym do normalnego, a wiec o srednije pb i odchyleniu...

def estimated_parameters(N: int, n: int) -> Tuple[float, float]:
    p = n / N
    sigma = math.sqrt(p * (1 - p) / N)
    return p, sigma

#jezeli zalozymy ze te dwie zmienne o rozkladzie normalnym sa niezalezne (poszczegolne proby Bernoulliego powinny byc
#niezalezne) to ich roznica rowniez powinna miec rozklad normalny o sredniej pb - pa i odchyleniu stadnardowym sqrt(sigma^2a+
#sigma^2b)

#teraz mozemy sprawdzic hipoteze zerowa, ktora mowi ze wartoscia pa i pb sa takie same

def a_b_test_statistic(N_A: int, n_A: int, N_B: int, n_B: int) -> float:
    p_A, sigma_A = estimated_parameters(N_A, n_A)
    p_B, sigma_B = estimated_parameters(N_B, n_B)
    return (p_B - p_A) / math.sqrt(sigma_A ** 2 + sigma_B ** 2)

z = a_b_test_statistic(1000, 200, 1000, 180)   
c = two_sided_p_value(z)       
print(z, c)



z2 = a_b_test_statistic(1000, 200, 1000, 150)   
c2 = two_sided_p_value(z2)
print(z2, c2)

#prawdopodobienstwo 0.25 oznacza ze srednie sa w 25% rowne i nie ma miedzy nimi zbyt duzej roznicyy
#drugi przypadek oznacza ze prawdopodobienstwo uzyskania tak duzej roznicy przy rownie skutecznych reklamach wynosi 0.003 czyli 
#reklamy nie sa sobie rowne i skutecznosc jednej jest wieksza

-1.1403464899034472 0.254141976542236
-2.948839123097944 0.003189699706216853


# wnioskowanie bayesowskie

In [ ]:
#jest to alternatywna meetoda wnioskowania, traktuje ona nieznane parametry jakby byly zmiennymi losowymi
#Analityk rozpoczyna prace od rozkladu a priori a pozniej korzysta z 
#zebranych w ten sposob danych i z twierdzenia BAYESA w celu uzyskania rozkladu a posteriori parametrow
#Okreslamy prawdopodobienstwo samych parametrow

In [ ]:
#jezeli nieznany parametr jest prawdopodobienstwem to
#korzystami z a priori rozkladu beta, w tym rozkladzie wszystkie prawdopodobienstwa przyjmuja wartosci od 0 do 1
def B(alpha: float, beta: float) -> float:
    """stala normalizacji zapewniajaca calkowite prawdopodobienstwo rowne 1"""
    return math.gamma(alpha) * math.gamma(beta) / math.gamma(alpha + beta)

def beta_pdf(x: float, alpha: float, beta: float) -> float:
    if x < 0 or x > 1:             #wszystkie wagi mieszcza sie w zakresie [0, 1]
        return 0
    return x ** (alpha - 1) * (1 - x) ** (beta - 1) / B(alpha, beta)

#ww rozklad, wysrodkowuje wagi w punkcie